#### Sunday, December 17, 2023

[Tutorial: Answering Multihop Questions with Agents](https://haystack.deepset.ai/tutorials/23_answering_multihop_questions_with_agents)

This notebook uses OpenAI. Currently at Used/Expired : $4.42 / $38.00

This all runs.

In [1]:
# only target the 4090 ...
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

# Tutorial: Answering Multihop Questions with Agents

- **Level**: Intermediate
- **Time to complete**: 10 minutes
- **Nodes Used**: `Agent`, `PromptNode`, `InMemoryDocumentStore`, `FARMReader` and `ExtractiveQAPipeline`
- **Goal**: After completing this tutorial, you will have learned how to use Agents to build a multi-hop question answering system with an `ExtractiveQAPipeline` as a tool
- **Prerequisites**: An [OpenAI API Key](https://help.openai.com/en/articles/4936850-where-do-i-find-my-secret-api-key)


## Overview

The [Agent](https://docs.haystack.deepset.ai/docs/agent) class uses a large language model (LLM) to make decisions and come up with the best next course of action. You can provide the `Agent` with a set of [`Tools`](https://docs.haystack.deepset.ai/docs/agent#tools) that it can choose to use to reach a result. At each iteration, the agent will pick a tool from the ones available to it. Based on the result, the Agent has two options: It will either decide to select a tool again and do another iteration, or it will decide that it has reached a conclusion and return the final answer.

In this tutorial, we will provide the Agent with just one tool to answer questions: a commonly used Haystack component, the `ExtractiveQAPipeline`.


## Preparing the Colab Environment

- [Enable GPU Runtime in Colab](https://docs.haystack.deepset.ai/docs/enabling-gpu-acceleration#enabling-the-gpu-in-colab)
- [Set logging level to INFO](https://docs.haystack.deepset.ai/docs/log-level)

## Installing Haystack

To start, let's install the latest release of Haystack with `pip`. In this tutorial, we'll use a [Hugging Face dataset](https://huggingface.co/datasets/Tuana/presidents) that has already been prepared as Haystack `Documents`, so we will install `datasets` too:

In [ ]:
# %%bash

# pip install --upgrade pip
# pip install farm-haystack[colab,inference]
# pip install datasets


### Enabling Telemetry 
Knowing you're using this tutorial helps us decide where to invest our efforts to build a better product but you can always opt out by commenting the following line. See [Telemetry](https://docs.haystack.deepset.ai/docs/telemetry) for more details.

In [2]:
from haystack.telemetry import tutorial_running

tutorial_running(23)

Set the logging level to INFO:

In [3]:
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)

## Create an Extractive QA Pipeline

Now, we will introduce an `ExtractiveQAPipeline` as a `Tool` to our `Agent`. To do so, we'll first write our documents about the presidents of the USA into a `DocumentStore` and then create our pipeline.

### 1) Write documents to the DocumentStore

In [4]:
from haystack.document_stores import InMemoryDocumentStore
from datasets import load_dataset

remote_dataset = load_dataset("Tuana/presidents", split="train")

document_store = InMemoryDocumentStore(use_bm25=True)
document_store.write_documents(remote_dataset)

# download time
# 8.2s

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/5529 [00:00<?, ? examples/s]

INFO - haystack.modeling.utils -  Using devices: CUDA:0 - Number of GPUs: 1
INFO - haystack.document_stores.base -  Duplicate Documents: Document with id '4ba0fe7553f54a0c06ca28f7379f7afc' already exists in index 'document'
INFO - haystack.document_stores.base -  Duplicate Documents: Document with id '4ba0fe7553f54a0c06ca28f7379f7afc' already exists in index 'document'
INFO - haystack.document_stores.base -  Duplicate Documents: Document with id '4ba0fe7553f54a0c06ca28f7379f7afc' already exists in index 'document'
INFO - haystack.document_stores.base -  Duplicate Documents: Document with id '78b1c282803ef0b7fd8cb6c2c6d0348' already exists in index 'document'
INFO - haystack.document_stores.base -  Duplicate Documents: Document with id '36b5dd40c2ecc832c4ea398b995ed16c' already exists in index 'document'
INFO - haystack.document_stores.base -  Duplicate Documents: Document with id '4ba0fe7553f54a0c06ca28f7379f7afc' already exists in index 'document'
INFO - haystack.document_stores.base 

### 2) Create an ExtractiveQAPipeline:
Let's define our retriever and reader to to use in an `ExtractiveQAPipeline`:

In [5]:
from haystack.nodes import EmbeddingRetriever, FARMReader
from haystack.pipelines import ExtractiveQAPipeline

retriever = EmbeddingRetriever(
    document_store=document_store, embedding_model="sentence-transformers/multi-qa-mpnet-base-dot-v1", use_gpu=True
)
document_store.update_embeddings(retriever=retriever)
reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)
presidents_qa = ExtractiveQAPipeline(reader=reader, retriever=retriever)

# 27.7s

INFO - haystack.modeling.utils -  Using devices: CUDA:0 - Number of GPUs: 1
INFO - haystack.nodes.retriever.dense -  Init retriever using embeddings of model sentence-transformers/multi-qa-mpnet-base-dot-v1
/usr/local/lib/python3.8/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
INFO - haystack.document_stores.memory -  Updating embeddings for 0 docs ...
Updating Embedding:   0%|          | 0/5400 [00:00<?, ? docs/s]

Batches:   0%|          | 0/169 [00:00<?, ?it/s]

Documents Processed: 10000 docs [00:23, 417.10 docs/s]          
INFO - haystack.modeling.utils -  Using devices: CUDA:0 - Number of GPUs: 1
INFO - haystack.modeling.utils -  Using devices: CUDA:0 - Number of GPUs: 1
INFO - haystack.modeling.model.language_model -   * LOADING MODEL: 'deepset/roberta-base-squad2' (Roberta)
INFO - haystack.modeling.model.language_model -  Auto-detected model language: english
INFO - haystack.modeling.model.language_model -  Loaded 'deepset/roberta-base-squad2' (Roberta model) from model hub.
INFO - haystack.modeling.utils -  Using devices: CUDA:0 - Number of GPUs: 1


### 3) Let's test the pipeline!

Now that you have an `ExtractiveQAPipeline`, go ahead and ask it a question about the presidents of the USA. 

Below, we're asking the question: "What year was the 1st president of the USA born?"

Notice how this is 2 questions in one. An extractive model will struggle to find the answer to this question unless the answer is phrased clearly in our documents. For example: "The first president of the USA was born in 1732".

On the other hand, it does well with a question such as "Who was the 1st president of the USA?"

In [6]:
from haystack.utils import print_answers

# result = presidents_qa.run("Who was the 1st president of the USA?")
result = presidents_qa.run("What year was the 1st president of the USA born?")

print_answers(result, "minimum")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  6.66 Batches/s]

'Query: What year was the 1st president of the USA born?'
'Answers:'
[   {   'answer': '1773',
        'context': 'enjamin Harrison V and Elizabeth (Bassett) Harrison, born '
                   'on February 9, 1773, at Berkeley Plantation, the home of '
                   'the Harrison family of Virginia on '},
    {   'answer': '^ Hoover',
        'context': 'k}.mw-parser-output '
                   '.reflist-lower-roman{list-style-type:lower-roman}\n'
                   '\n'
                   '^ Hoover later became the first president born west of the '
                   'Mississippi River, a'},
    {   'answer': 'Jefferson (1801)',
        'context': ' a series aboutUnited Statespresidential transitions\n'
                   '\n'
                   'Transitions\n'
                   '\n'
                   'Jefferson (1801)\n'
                   'Taylor (1848–49)\n'
                   'Lincoln (1860–61)\n'
                   'Taft (1908–09)\n'
                   'Wilson (1912–13)'

## Create an Agent with the `ExtractiveQAPipeline` as a `Tool`
### 1) To create the Agent, we'll make use of an Open AI model. So first, provide your Open AI key:

In [7]:
import os
from getpass import getpass

api_key = os.getenv("OPENAI_API_KEY", None) or getpass("Enter OpenAI API key:")

### 2) Initialize the Agent 

The `Agent` needs to determine the next best course of action at each iteration. It does this by using an LLM, and a prompt designed specially for this use case. Our `Agent` uses a `PromptNode` with the default ["zero-shot-react" `PromptTemplate` ](https://github.com/deepset-ai/haystack/blob/444a3116c42d2c8852d27aa8093ac92c8e85ab88/haystack/nodes/prompt/prompt_node.py#L337). 

Here, let's define an `Agent` that uses the `text-davinci-003` model by OpenAI.

In [9]:
from haystack.agents import Agent
from haystack.nodes import PromptNode

prompt_node = PromptNode(model_name_or_path="text-davinci-003", api_key=api_key, stop_words=["Observation:"])
agent = Agent(prompt_node=prompt_node)

/usr/local/lib/python3.8/dist-packages/haystack/nodes/prompt/prompt_template.py:444: UserWarning: You're using a legacy prompt template 'zero-shot-react', we strongly suggest you use prompts from the official Haystack PromptHub: https://prompthub.deepset.ai/
  warnings.warn(


### 3) Provide the Agent with a Tool
Next, let's add our `ExtractiveQAPipeline` into the Agent's arsenal. The Agent will then be able to use this pipeline when it decides it could be useful.

To do so, let's define a tool and make sure to give it a description. The exact wording of your description matters a lot here. The agent uses it to understand in which cases it should pick this tool. If the agent fails to pick the right tool, adjusting the description might help.

In [10]:
from haystack.agents import Tool

search_tool = Tool(
    name="USA_Presidents_QA",
    pipeline_or_node=presidents_qa,
    description="useful for when you need to answer questions related to the presidents of the USA.",
    output_variable="answers",
)
agent.add_tool(search_tool)

### 4) Ask a question!


In [11]:
result = agent.run("What year was the 1st president of the USA born?")

print(result["transcript"].split("---")[0])


Agent zero-shot-react started with {'query': 'What year was the 1st president of the USA born?', 'params': None}
 find out the name of the 1st president of the USA. 
Tool: USA_Presidents_QA 
Tool Input: List presidents


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Inferencing Samples: 100%|██████████| 4/4 [00:00<00:00, 10.81 Batches/s]

Observation: William Howard Taft
Thought: 

 Now that I have the name of the president, I can use the same tool to get the year he was born. 
Tool: USA_Presidents_QA 
Tool Input: William Howard Taft



Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 33.46 Batches/s]

Observation: Yale College photograph of Taft, c. 1878
William Howard Taft was born September 15, 1857
Thought: 

 Ok, now I have the year he was born.
Final Answer: 1857 find out the name of the 1st president of the USA. 
Tool: USA_Presidents_QA 
Tool Input: List presidents

Observation: William Howard Taft
Thought: Now that I have the name of the president, I can use the same tool to get the year he was born. 
Tool: USA_Presidents_QA 
Tool Input: William Howard Taft


Observation: Yale College photograph of Taft, c. 1878
William Howard Taft was born September 15, 1857
Thought: Ok, now I have the year he was born.
Final Answer: 1857


Congratulations! 🎉 You've used an Agent that can use an extractive model iteratively, to arrive at a final answer to a multi-hop question!!!